In [1]:
from solid import *
from solid.utils import *  # Not required, but the utils module is useful
import viewscad

In [2]:
r = viewscad.Renderer()

In [29]:
# Variables
BASE_DIAMETER = 70
BASE_HEIGHT = 78
CUP_DIAMETER = 87
CUP_HEIGHT = 116
INSERT_HEIGHT = 35
CUP_ADDL_RADIUS = 1
PLATE_HEIGHT = 5
NOTCH_WIDTH = 20
WALL_THICKNESS = 3.2
SEGMENTS = 60
DRAINAGE_DIAMETER = BASE_DIAMETER/2

# Derived
BASE_RADIUS = (
    BASE_DIAMETER / 2
) - 0.5  # This fits INTO the cup holder, so lets try for smaller
BASE_HEIGHT_NO_INSERT = BASE_HEIGHT - INSERT_HEIGHT  # The insert comes from the top
CUP_RADIUS = (
    CUP_DIAMETER / 2
) + 1  # This should contain the cup, so lets try for larger


base = difference()(
    cylinder(r=BASE_RADIUS, h=BASE_HEIGHT_NO_INSERT, segments=SEGMENTS),
    cylinder(
        r=BASE_RADIUS - WALL_THICKNESS, h=BASE_HEIGHT_NO_INSERT, segments=SEGMENTS
    ),
)


top = up(BASE_HEIGHT_NO_INSERT + PLATE_HEIGHT)(
    difference()(
        cylinder(
            r=CUP_RADIUS + WALL_THICKNESS + CUP_ADDL_RADIUS,
            h=CUP_HEIGHT / 2,
            segments=SEGMENTS,
        ),
        cylinder(r=CUP_RADIUS + CUP_ADDL_RADIUS, h=CUP_HEIGHT / 2, segments=SEGMENTS),
    )
)
plate = up(BASE_HEIGHT_NO_INSERT)(
    cylinder(
        r=CUP_RADIUS + WALL_THICKNESS + CUP_ADDL_RADIUS,
        h=PLATE_HEIGHT,
        segments=SEGMENTS,
    )
)

drainage = cylinder(d=DRAINAGE_DIAMETER, h=BASE_HEIGHT_NO_INSERT + PLATE_HEIGHT)

scadfile = import_scad("../common/roundedrectdiff.scad")

# old
handle_notch = translate((CUP_RADIUS - 5, 0, BASE_HEIGHT_NO_INSERT + PLATE_HEIGHT))(
    cube(size=[NOTCH_WIDTH, NOTCH_WIDTH, CUP_HEIGHT], center=True)
)

# new
handle_notch = translate(
    (
        CUP_RADIUS - WALL_THICKNESS,
        0,
        BASE_HEIGHT_NO_INSERT + PLATE_HEIGHT + (CUP_HEIGHT / 4),
    )
)(
    rotate([90, 270, 90])(
        linear_extrude(WALL_THICKNESS * 3)(
            scadfile.rounded_subtraction_square_top(CUP_HEIGHT / 2, NOTCH_WIDTH, 5)
        )
    )
)
handle_notch2 = rotate([0,0,(360/3)])(handle_notch)
handle_notch3 = rotate([0,0,(360/3)*2])(handle_notch)


complete = base + (plate - drainage) + (top - (handle_notch + handle_notch2 + handle_notch3))


In [30]:
r.render(complete)

Geometries in cache: 31
Geometry cache size in bytes: 87664
CGAL Polyhedrons in cache: 7
CGAL cache size in bytes: 2649680
Total rendering time: 0:00:10.992
   Top level object is a 3D object:
   Simple:        yes
   Vertices:      618
   Halfedges:    1854
   Edges:         927
   Halffacets:    626
   Facets:        313
   Volumes:         2
/Users/peter/miniconda3/envs/py-open-scad/lib/python3.8/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [19]:
print(scad_render(complete))

use </Users/peter/projects/py-open-scad/cupholder/../common/roundedrectdiff.scad>


union() {
	difference() {
		cylinder($fn = 60, h = 43, r = 34.5000000000);
		cylinder($fn = 60, h = 43, r = 31.3000000000);
	}
	translate(v = [0, 0, 43]) {
		cylinder($fn = 60, h = 5, r = 48.7000000000);
	}
	difference() {
		translate(v = [0, 0, 48]) {
			difference() {
				cylinder($fn = 60, h = 58.0000000000, r = 48.7000000000);
				cylinder($fn = 60, h = 58.0000000000, r = 45.5000000000);
			}
		}
		color(alpha = 1.0000000000, c = "red") {
			translate(v = [41.3000000000, 0, 77.0000000000]) {
				rotate(a = [90, 270, 90]) {
					linear_extrude(height = 9.6000000000) {
						rounded_subtraction_square_top(height = 58.0000000000, radius = 5, width = 20);
					}
				}
			}
		}
	}
}


In [31]:
scad_render_to_file(complete, "cupholder3.scad")

'/Users/peter/projects/py-open-scad/cupholder/cupholder3.scad'

In [ ]:
!openscad 